In [ ]:
import os

In [ ]:
__file__ = "/home/kwat/github/sample_template/code/process_sample.ipynb"

In [ ]:
project_directory_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

code_directory_path = os.path.join(project_directory_path, "code")

input_directory_path = os.path.join(project_directory_path, "input")

output_directory_path = os.path.join(project_directory_path, "output")

In [ ]:
# reference_directory_path = os.path.join(input_directory_path, 'reference')
reference_directory_path = "/media/kwat/CarrotCake/data/"

sample_directory_path = os.path.join(input_directory_path, "sample")

summary_directory_path = os.path.join(output_directory_path, "summary")

In [ ]:
import kraft

In [ ]:
project_json = kraft.read_json(os.path.join(project_directory_path, "project.json"))

project_json

In [ ]:
output_germ_dna_directory_path = os.path.join(output_directory_path, "germ_dna")

output_soma_dna_directory_path = os.path.join(output_directory_path, "soma_dna")

output_soma_rna_directory_path = os.path.join(output_directory_path, "soma_rna")

In [ ]:
if "germ_dna.1" in project_json and "germ_dna.2" in project_json:

    print("Processing germ DNA ...")

    kraft.run_command(
        "{}/process_germ_dna.sh {} {} {} {} {} exome {}".format(
            code_directory_path,
            reference_directory_path,
            project_json["n_job"],
            project_json["gb_memory"],
            project_json["germ_dna.1"],
            project_json["germ_dna.2"],
            output_germ_dna_directory_path,
        )
    )

In [ ]:
if "soma_dna.1" in project_json and "soma_dna.2" in project_json:

    print("Processing soma DNA ...")

    kraft.run_command(
        "{}/process_soma_dna.sh {} {} {} {} {} {} {} exome {}".format(
            code_directory_path,
            reference_directory_path,
            project_json["n_job"],
            project_json["gb_memory"],
            project_json["germ_dna.1"],
            project_json["germ_dna.2"],
            project_json["soma_dna.1"],
            project_json["soma_dna.2"],
            output_soma_dna_directory_path,
        )
    )

In [ ]:
if "soma_rna.1" in project_json and "soma_rna.2" in project_json:

    print("Processing soma RNA ...")

    kraft.run_command(
        "{}/process_soma_rna.sh {} {} {} {} {} {}".format(
            code_directory_path,
            reference_directory_path,
            project_json["n_job"],
            project_json["gb_memory"],
            project_json["soma_rna.1"],
            project_json["soma_rna.2"],
            output_soma_rna_directory_path,
        )
    )

In [ ]:
import shutil

In [ ]:
if os.path.isdir(summary_directory_path):

    shutil.rmtree(summary_directory_path)

os.mkdir(os.path.join(summary_directory_path))

In [ ]:
if os.path.isdir(output_germ_dna_directory_path):

    germ_variant_n = kraft.make_variant_n_from_vcf_file_path(
        os.path.join(output_germ_dna_directory_path, "snpeff", "variant.vcf.gz")
    )

    germ_variant_n.to_csv(
        os.path.join(summary_directory_path, "germ_dna.variant_n.tsv"),
        sep="\t",
        header=True,
    )

    print(germ_variant_n)

In [ ]:
if os.path.isdir(output_soma_dna_directory_path):

    soma_variant_n = kraft.make_variant_n_from_vcf_file_path(
        os.path.join(output_soma_dna_directory_path, "snpeff", "variant.vcf.gz")
    )

    soma_variant_n.to_csv(
        os.path.join(summary_directory_path, "soma_dna.variant_n.tsv"),
        sep="\t",
        header=True,
    )

    print(soma_variant_n)

In [ ]:
import pandas as pd

In [ ]:
if os.path.isdir(output_soma_rna_directory_path):

    enst_tpm = pd.read_csv(
        os.path.join(
            output_soma_rna_directory_path, "kallisto", "transcriptome", "abundance.tsv"
        ),
        sep="\t",
        index_col=0,
    )["tpm"].sort_values(ascending=False)

    enst_tpm.index.name = "ENST"

    enst_tpm.name = "TPM"

    enst_tpm.to_csv(
        os.path.join(summary_directory_path, "enst_tpm.tsv"), sep="\t", header=True
    )

    print(enst_tpm)

    enst_gene_name = pd.read_csv(
        os.path.join(reference_directory_path, "grch", "enst_gene_name.tsv"), sep="\t"
    )

    enst_gene_name = dict(
        zip(enst_gene_name["Transcript stable ID version"], enst_gene_name["Gene name"])
    )

    gene_tpm = pd.Series(
        enst_tpm.values,
        index=enst_tpm.index.map(lambda enst: enst_gene_name.get(enst, enst)),
    )

    size_before = gene_tpm.size

    gene_tpm = gene_tpm.groupby(by=gene_tpm.index).median().sort_values(ascending=False)

    print("Size: {} =(groupby)=> {}".format(size_before, gene_tpm.size))

    gene_tpm.index.name = "Gene"

    gene_tpm.name = "Median TPM"

    gene_tpm.to_csv(
        os.path.join(summary_directory_path, "gene_tpm.tsv"), sep="\t", header=True
    )

    print(gene_tpm)

    virus_id_tpm = pd.read_csv(
        os.path.join(
            output_soma_rna_directory_path, "kallisto", "virus", "abundance.tsv"
        ),
        sep="\t",
        index_col=0,
    )["tpm"].sort_values(ascending=False)

    virus_id_tpm.index.name = "Virus ID"

    virus_id_tpm.name = "TPM"

    virus_id_tpm.to_csv(
        os.path.join(summary_directory_path, "virus_id_tpm.tsv"), sep="\t", header=True
    )

    print(virus_id_tpm)

    virus_id_name = pd.read_csv(
        os.path.join(reference_directory_path, "virus", "sequences.csv"), index_col=0
    )["Species"].to_dict()

    virus_name_tpm = pd.Series(
        virus_id_tpm.values,
        index=virus_id_tpm.index.map(
            lambda virus_id: virus_id_name.get(virus_id, virus_id)
        ),
    )

    size_before = virus_name_tpm.size

    virus_name_tpm = (
        virus_name_tpm.groupby(by=virus_name_tpm.index)
        .max()
        .sort_values(ascending=False)
    )

    print("Size: {} =(groupby)=> {}".format(size_before, virus_name_tpm.size))

    virus_name_tpm.index.name = "Virus Name"

    virus_name_tpm.name = "Max TPM"

    virus_name_tpm.to_csv(
        os.path.join(summary_directory_path, "virus_name_tpm.tsv"),
        sep="\t",
        header=True,
    )

    print(virus_name_tpm)